# Read summary file of simulations and plot results   

After all simulations (`SIXTE`+ `xifusim`) a file is saved with all the information.   
This notebook reads that file and plots summaries:   

1. **Histogram** of fraction of lost photons: has a 'normal' distribution? Can I assign a median value and a std deviation for that flux?   
   It is important to establish a simulation time that produces enough number of close pairs that could give rise to pile-up. Here we check that the fractions obtained in the different simulations are 'normally' distributed so that we can finally obtain a mean value   

In [ ]:
# import modules for ploting and data manipulation
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.visualization import hist


In [ ]:
flux_mcrab = f"{0.5:.3f}mCrab"
summ_file = f"info_nofilt_infoc_global_{flux_mcrab}.csv"
print(f"Reading file: {summ_file}")
#exposure = 4331
# read csv file
df = pd.read_csv(summ_file)
print(df.head())

In [ ]:
# Read columns "simulation" and "fraction[%lost]" for the given exposure time
# get exposure time from table
exposure = df['exposure[s]'].values[0]
filter = df['filter'].values[0]
focus = df['focus'].values[0]
Nimpacts = df['Nimpacts'].values[0]
sim = df["simulation"]
frac = df["fraction_lost[%]"]
# plot a histogram of the fraction of lost photons for each simulation
hist(frac, bins='scott', color="blue", alpha=0.7)
plt.xlabel("Fraction of lost photons [%]")
plt.ylabel("Number of simulations")
plt.title(f"Histogram of fraction of lost photons ({len(sim)} simulations)")
# print info about flux, exposure time, Nimpacts, filter and focus
mean = np.mean(frac)
std = np.std(frac)
pm_str = u"\u00B1"
plt.text(0.6, 0.65, f"Flux: {flux_mcrab}\nExposure time: {exposure} s\nNimpacts: {Nimpacts}\nFilter: {filter}\nFocus: {focus}\nMean: {mean:.3f} {pm_str} {std:.3f}%", fontsize=12, transform=plt.gca().transAxes)
plt.show()
#plt.hist(frac, bins=10, color="blue", alpha=0.7)

In [ ]:
mean_frac = np.mean(frac)
std_frac = np.std(frac)
print("Mean fraction of lost photons: ", mean_frac)
print("Standard deviation of fraction of lost photons: ", std_frac)


In [ ]:
square = std_frac**2
print("Variance of fraction of lost photons: ", square)


In [ ]:
# do a normality test
from scipy.stats import shapiro
stat, p = shapiro(frac)
print("Shapiro-Wilk test statistic: ", stat)
print("Shapiro-Wilk test p-value: ", p)
# print a conclusion about the normality of the data
if p > 0.05:
    print("The fraction of lost photons is normally distributed")


In [ ]:
# do an anderson-darling test for normality
from scipy.stats import anderson
result = anderson(frac)
print("Anderson-Darling test statistic: ", result.statistic)
print("Anderson-Darling test critical values: ", result.critical_values)
print("Anderson-Darling test significance levels: ", result.significance_level)
print("Anderson-Darling test p-value: ", result.significance_level[2])
# print a conclusion about normality
if result.statistic < result.critical_values[2]:
    print("The fraction of lost photons is normally distributed")
